# Sandbox Exporter Demo

The [sandbox_exporter](https://github.com/usdot-its-jpo-data-portal/sandbox_exporter) package allows users to load, query, and export data from ITS DataHub's sandbox efficiently by providing a wrapper around the s3select package and providing additional functionality for interacting with the ITS DataHub data. This notebook contains short instructions and sample code for how to use this tool.

### Installation

1. Download the script by cloning the module's [code repository on GitHub](https://github.com/usdot-its-jpo-data-portal/wzdx_sandbox). You can do so by running one of the following in command line. If unfamiliar with how to clone a repository, follow the [official GitHub guide](https://help.github.com/en/articles/cloning-a-repository).
    - via HTTP: `git clone https://github.com/usdot-its-jpo-data-portal/sandbox_exporter.git`
    - via SSH (if using 2-factor authentication): `git clone git@github.com:usdot-its-jpo-data-portal/sandbox_exporter.git`
2. Navigate into the repository folder by entering `cd sandbox_exporter` in command line.
3. Run `pip install -e .` to install the sandbox_exporter Python package.
4. Install the required packages by running `pip install -r requirements.txt`.

### Use cases
- Users can now pass a list of fields into the function and get back only those fields
- Users have the following output format options: 
  - return object array
  - object generator
  - write to csv file
  - write to newline json file
- User have the option to turn off print statements (added verbose flag)
- Now uses S3 Select to query the data, which is more speed and cost-efficient
- Utilizing a package that is also able to show the estimated cost associated with each query

## Import and instantiate the SandboxExporter

In [1]:
from sandbox_exporter.exporter import SandboxExporter

bucket = 'usdot-its-cvpilot-public-data'
aws_profile = 'default'

# instantiate verbose exporter
exporter = SandboxExporter(aws_profile=aws_profile, verbose=True, bucket=bucket)

# instantiate quiet exporter
exporter_q = SandboxExporter(aws_profile=aws_profile, verbose=False, bucket=bucket)

In [2]:
res = exporter.get_record_generator(sdate='2020-02-01', 
                                 edate='2020-02-05', 
                                 pilot='wydot', 
                                 message_type='BSM',
                                   limit=1)
list(res)

[{'metadata': {'bsmSource': 'RV',
   'logFileName': 'rxMsg_1580730164_2620:31:40e0:840::1.csv',
   'recordType': 'rxMsg',
   'securityResultCode': 'success',
   'receivedMessageDetails': {'locationData': {'latitude': '0',
     'longitude': '0',
     'elevation': '0',
     'speed': '0',
     'heading': '0'},
    'rxSource': 'RV'},
   'payloadType': 'us.dot.its.jpo.ode.model.OdeBsmPayload',
   'serialId': {'streamId': 'a61ed1e5-46b5-4aab-b9c0-5c04a655332c',
    'bundleSize': 438,
    'bundleId': 379664,
    'recordId': 1,
    'serialNumber': 19554482},
   'odeReceivedAt': '2020-02-03T11:51:07.116Z',
   'schemaVersion': 6,
   'recordGeneratedAt': '2020-02-01T00:37:14.291Z',
   'recordGeneratedBy': 'OBU',
   'sanitized': True},
  'payload': {'dataType': 'us.dot.its.jpo.ode.plugin.j2735.J2735Bsm',
   'data': {'coreData': {'msgCnt': 111,
     'id': '57303235',
     'secMark': 14299,
     'position': {'latitude': 41.5325156,
      'longitude': -109.3813391,
      'elevation': 1867.6},
     'a

## Check Number of Records

In [3]:
# check number of WYDOT and THEA BSM records between 2020-02-01 - 2020-02-05
wydot_count = exporter.get_count(sdate='2020-02-01', 
                                 edate='2020-02-05', 
                                 pilot='wydot', 
                                 message_type='BSM')
print('{} records from WYDOT\n'.format(wydot_count))

thea_count = exporter_q.get_count(sdate='2020-02-01', 
                                 edate='2020-02-05', 
                                 pilot='thea', 
                                 message_type='BSM')
print('{} records from THEA\n'.format(thea_count))

Files processed: 16/16  Records matched: 13510  Bytes scanned: 26 MB
Cost for data scanned: $0.00
Cost for data returned: $0.00
Cost for SELECT requests: $0.00
Total cost: $0.00
13510 records from WYDOT

1152 records from THEA



# Retrieve 5 sample records

Retrieve 5 sample BSM records from each pilot and compare the metadata schema of the record from each pilot.

In [4]:
# retrieve all fields from 5 records
wydot_records = exporter_q.get_records(sdate='2020-02-01', 
                                     edate='2020-02-05', 
                                     pilot='wydot', 
                                     message_type='BSM',
                                     limit=5)
thea_records = exporter_q.get_records(sdate='2020-02-01', 
                                     edate='2020-02-05', 
                                     pilot='thea', 
                                     message_type='BSM',
                                     limit=5)

In [5]:
thea_records[0]['metadata']

{'schemaVersion': 1,
 'recordGeneratedBy': 'rsu',
 'recordGeneratedAt': '2020-02-03 00:24:02.668 [ET]',
 'logFileName': '2020_2_3_0_thea2.csv',
 'kind': 'in',
 'bsmSource': 'RV',
 'psid': '32',
 'RSUID': 'thea2',
 'externalID': 'NA',
 'dataType': 'bsm'}

In [6]:
wydot_records[0]['metadata']

{'bsmSource': 'RV',
 'logFileName': 'rxMsg_1580517741_2620:31:40e0:843::1.csv',
 'recordType': 'rxMsg',
 'securityResultCode': 'spduParsingCertificateNotFound',
 'receivedMessageDetails': {'locationData': {'latitude': '0',
   'longitude': '0',
   'elevation': '0',
   'speed': '0',
   'heading': '0'},
  'rxSource': 'RV'},
 'payloadType': 'us.dot.its.jpo.ode.model.OdeBsmPayload',
 'serialId': {'streamId': 'a61ed1e5-46b5-4aab-b9c0-5c04a655332c',
  'bundleSize': 458,
  'bundleId': 378258,
  'recordId': 90,
  'serialNumber': 19248742},
 'odeReceivedAt': '2020-02-01T00:48:05.376Z',
 'schemaVersion': 6,
 'recordGeneratedAt': '2020-02-01T00:41:33.296Z',
 'recordGeneratedBy': 'OBU',
 'sanitized': True}

In [7]:
wydot_count = exporter.get_count(
    sdate='2020-02-01', 
    edate='2020-02-05', 
    pilot='wydot', 
    message_type='BSM',
    where="s.metadata.bsmSource='RV' and s.payload.data.coreData.speed < 15"
)
print('{} records from WYDOT with speed < 15 \n'.format(wydot_count))

thea_count = exporter.get_count(
    sdate='2020-02-01', 
    edate='2020-02-05', 
    pilot='thea', 
    message_type='BSM',
    where="s.metadata.bsmSource='RV' and s.payload.data.coreData.speed < '15'"
)
print('{} records from THEA with speed < 15 \n'.format(thea_count))

Files processed: 16/16  Records matched: 8913  Bytes scanned: 26 MB
Cost for data scanned: $0.00
Cost for data returned: $0.00
Cost for SELECT requests: $0.00
Total cost: $0.00
8913 records from WYDOT with speed < 15 

Files processed: 8/8  Records matched: 44  Bytes scanned: 2 MB
Cost for data scanned: $0.00
Cost for data returned: $0.00
Cost for SELECT requests: $0.00
Total cost: $0.00
44 records from THEA with speed < 15 



# Retrieve records from a date range in an json object array 

In [8]:
# retrieve selected fields from 5 records
records = exporter.get_records(sdate='2020-01-22', 
                               edate='2020-01-25',
                               pilot='wydot', 
                               message_type='BSM', 
                               output_fields='s.payload.data.coreData.speed',
                               limit=5
                              )
records

Files processed: 0/23  Records matched: 5  Bytes scanned: 0 B
Cost for data scanned: $0.00
Cost for data returned: $0.00
Cost for SELECT requests: $0.00
Total cost: $0.00


[{'speed': 17.3},
 {'speed': 17.3},
 {'speed': 17.3},
 {'speed': 17.3},
 {'speed': 17.3}]

In [9]:
# export 100 records to a CSV file, limiting the fields to export and filtering by certain variables 
exporter.export_to_file(sdate='2020-01-22', 
                        pilot='wydot', message_type='BSM',
                        where="s.metadata.bsmSource='RV' and s.payload.data.coreData.speed < 15",
                        limit=100,
                        output_fields='s.metadata',csv=True)

Wrote 100 recs to wydot_bsm_2020012200_2020012300_0.csv
Output files:
wydot_bsm_2020012200_2020012300_0.csv

Files processed: 2/7  Records matched: 100  Bytes scanned: 906 KB
Cost for data scanned: $0.00
Cost for data returned: $0.00
Cost for SELECT requests: $0.00
Total cost: $0.00
Process took 0.02312944730122884 minutes
============END============


In [10]:
# export 100 records to a JSON newline file, limiting the fields to export and filtering by certain variables
exporter_q.export_to_file(sdate='2020-01-22', 
                          pilot='wydot', message_type='BSM', limit=5,
                          output_fields='s.metadata,s.payload.data.coreData.speed',
                          where="s.metadata.bsmSource='RV' and s.payload.data.coreData.speed < 15",
                        csv=False)

In [11]:
# create a record generator
generator = exporter.get_record_generator(sdate='2020-01-22', 
                       pilot='wydot', message_type='BSM', limit=5)

In [12]:
generator

<generator object S3Helper.select at 0x10a1a9e58>

In [12]:
for i in generator:
    print(i)
    break

{'metadata': {'bsmSource': 'RV', 'logFileName': 'rxMsg_1579659546_2620:31:40e0:839::1.csv', 'recordType': 'rxMsg', 'securityResultCode': 'success', 'receivedMessageDetails': {'locationData': {'latitude': '0', 'longitude': '0', 'elevation': '0', 'speed': '0', 'heading': '0'}, 'rxSource': 'RV'}, 'payloadType': 'us.dot.its.jpo.ode.model.OdeBsmPayload', 'serialId': {'streamId': 'a61ed1e5-46b5-4aab-b9c0-5c04a655332c', 'bundleSize': 395, 'bundleId': 311244, 'recordId': 3, 'serialNumber': 14138016}, 'odeReceivedAt': '2020-01-22T02:33:02.161Z', 'schemaVersion': 6, 'recordGeneratedAt': '2020-01-22T00:36:09.599Z', 'recordGeneratedBy': 'OBU', 'sanitized': True}, 'payload': {'dataType': 'us.dot.its.jpo.ode.plugin.j2735.J2735Bsm', 'data': {'coreData': {'msgCnt': 100, 'id': '57303235', 'secMark': 9600, 'position': {'latitude': 41.5282506, 'longitude': -109.4228493, 'elevation': 1855.7}, 'accelSet': {'accelLat': -9.95, 'accelLong': -0.09, 'accelVert': -0.06, 'accelYaw': 0.52}, 'accuracy': {'semiMajor